In [15]:
!python3.10 -m pip install fastkml GDAL geopandas fiona pandarallel tqdm

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# -----------------------------------------------------------------------------
# Initialization: Configure parallel processing
# -----------------------------------------------------------------------------

import sys
sys.path.append('..')

from baseline.utilities import *

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.ops import unary_union
import fiona
import json
from tqdm import tqdm
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=False, nb_workers=8)

fiona.drvsupport.supported_drivers['libkml'] = 'rw'
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'

kml_path = '../baseline/Building_Footprint.kml'

# Modo de ejecución: 'submission' o 'train'
MODE = 'train'

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [17]:
# -----------------------------------------------------------------------------
# Load KML File: Read building footprints and clean data
# -----------------------------------------------------------------------------

bld_footprint = (
    gpd.read_file(kml_path, driver='libkml')
    .replace('', np.nan)
)
print(bld_footprint.columns)

# Drop columns with all NaN values 
bld_footprint = bld_footprint.dropna(axis=1, how='all')
print(f"{bld_footprint.shape=}")
display(bld_footprint)

print(bld_footprint.crs)


Index(['Name', 'Description', 'geometry'], dtype='object')
bld_footprint.shape=(9436, 1)


,geometry
0,"MULTIPOLYGON (((-73.91903 40.8482, -73.91933 4..."
1,"MULTIPOLYGON (((-73.92195 40.84963, -73.92191 ..."
2,"MULTIPOLYGON (((-73.9205 40.85011, -73.92045 4..."
3,"MULTIPOLYGON (((-73.92056 40.8514, -73.92053 4..."
4,"MULTIPOLYGON (((-73.91234 40.85218, -73.91247 ..."
...,...
9431,"MULTIPOLYGON (((-73.95267 40.77923, -73.95254 ..."
9432,"MULTIPOLYGON (((-73.94964 40.77613, -73.94931 ..."
9433,"MULTIPOLYGON (((-73.9521 40.7688, -73.95174 40..."
9434,"MULTIPOLYGON (((-73.9523 40.75904, -73.95246 4..."


EPSG:4326


In [18]:
# -----------------------------------------------------------------------------
# Process Geometries: Convert to single polygons and compute areas
# -----------------------------------------------------------------------------

bld_footprint['geometry'] = bld_footprint['geometry'].apply(lambda x: unary_union(x))
bld_footprint = bld_footprint.to_crs(epsg=3395)

bld_footprint['area'] = bld_footprint['geometry'].area

bld_footprint = bld_footprint.to_crs(epsg=4326)

# bld_footprint.to_file('../data/other/bf.json', driver='GeoJSON')
display(bld_footprint)

# Get bbox from all GeoDataFrame
bbox = bld_footprint.total_bounds

,geometry,area
0,"POLYGON ((-73.91903 40.8482, -73.91933 40.8479...",1080.601783
1,"POLYGON ((-73.92195 40.84963, -73.92191 40.849...",166.114638
2,"POLYGON ((-73.9205 40.85011, -73.92045 40.8501...",246.325998
3,"POLYGON ((-73.92056 40.8514, -73.92053 40.8514...",138.914032
4,"POLYGON ((-73.91234 40.85218, -73.91247 40.852...",376.844794
...,...,...
9431,"POLYGON ((-73.95267 40.77923, -73.95254 40.779...",4328.841543
9432,"POLYGON ((-73.94964 40.77613, -73.94931 40.776...",15754.867830
9433,"POLYGON ((-73.9521 40.7688, -73.95174 40.76931...",16565.644435
9434,"POLYGON ((-73.9523 40.75904, -73.95246 40.7590...",2687.606658


In [19]:
# -----------------------------------------------------------------------------
# Load External NYC Buildings Data
# -----------------------------------------------------------------------------

# Source: https://nycmaps-nyc.hub.arcgis.com/datasets/nyc::building/about
bv = gpd.read_file('../baseline/BUILDING_view_-5690770882456580009.geojson')

bv = bv.to_crs(epsg=3395)
bv['area'] = bv['geometry'].area
bv = bv.to_crs(epsg=4326)

bv = bv.cx[bbox[0]:bbox[2], bbox[1]:bbox[3]].reset_index(drop=True)

bv_csv = (
    pd.read_csv('../baseline/BUILDING_view_7607496916235021567.csv')
    [['DOITT ID', 'Area', 'Length']]
)

bv = bv.join(bv_csv.set_index('DOITT ID'), on='DOITT_ID')

# https://github.com/CityOfNewYork/nyc-geo-metadata/blob/main/Metadata/Metadata_BuildingFootprints.md

bv = bv[['CONSTRUCTION_YEAR', 'FEATURE_CODE', 'GROUND_ELEVATION', 'HEIGHT_ROOF', 'LAST_EDITED_DATE', 'LAST_STATUS_TYPE', 'geometry', 'area', 'Area', 'Length']]

bv = bv.loc[bv['CONSTRUCTION_YEAR'] < 2021].reset_index(drop=True)     # ! Check consistency of results

bv['height_per_squared_meter'] = bv['HEIGHT_ROOF'] / bv['area']

print(f"{bv.shape=}")
bv = bv[bv['geometry'].apply(lambda x: bld_footprint['geometry'].intersects(x).any())].reset_index(drop=True)
print(f"{bv.shape=}")


bv.shape=(89519, 11)
bv.shape=(42951, 11)


In [20]:
# -----------------------------------------------------------------------------
# Load Training or Submission Data and Generate Buffers
# -----------------------------------------------------------------------------

if MODE == 'train':
    ground_df = pd.read_csv('../baseline/Training_data_uhi_index.csv')
elif MODE == 'submission':
    ground_df = pd.read_csv("../baseline/Submission_template.csv")
else:
    raise ValueError("MODE should be either 'train' or 'submission")

ground_df.columns = ground_df.columns.str.lower()

dataset = ground_df[['longitude', 'latitude']]
dataset['geometry'] = gpd.points_from_xy(dataset['longitude'], dataset['latitude'])

geodataset = gpd.GeoDataFrame(dataset, crs='EPSG:4326')     # , random_state=42     .sample(200, ignore_index=True)

radius_list = json.loads(open('./data/radius_list.json', 'r').read())['radius_list']


In [21]:
# -----------------------------------------------------------------------------
# Spatial Analysis: Compute Intersections with Buffers
# -----------------------------------------------------------------------------


for radius_meter in tqdm(radius_list, total=len(radius_list), desc='Radius Areas'):
    geodataset = geodataset.to_crs(epsg=3395)

    geodataset[f'buffer_{radius_meter}m'] = (
        geodataset.apply(lambda x: buffer_meters(x, radius_meter), axis=1)
        .set_crs(3395)
    )
    geodataset[f'buffer_{radius_meter}m_area'] = geodataset[f'buffer_{radius_meter}m'].area
    geodataset[f'buffer_{radius_meter}m'] = geodataset[f'buffer_{radius_meter}m'].to_crs(epsg=4326)

    # * EY .kml file Building Footprints
    intersecting_squares = gpd.sjoin(bld_footprint, 
                            gpd.GeoDataFrame(geometry=geodataset[f'buffer_{radius_meter}m'], crs=bld_footprint.crs), 
                            predicate="intersects",
                            how='inner'
                            ).drop_duplicates(subset=['index_right', 'geometry'])

    squares_gb = intersecting_squares.groupby('index_right')['area']

    geodataset[f"kml_max_areas_{radius_meter}m"] = geodataset.index.map(squares_gb.max()).fillna(0)
    geodataset[f"kml_min_areas_{radius_meter}m"] = geodataset.index.map(squares_gb.min()).fillna(0)
    geodataset[f"kml_mean_areas_{radius_meter}m"] = geodataset.index.map(squares_gb.mean()).fillna(0)
    geodataset[f"kml_sum_areas_{radius_meter}m"] = geodataset.index.map(squares_gb.sum()).fillna(0)
    geodataset[f"kml_count_buildings_{radius_meter}m"] = geodataset.index.map(squares_gb.count()).fillna(0)

    # * NYC Buildings
    intersecting_buildings = gpd.sjoin(bv[['Area', 'Length', 'geometry', 'GROUND_ELEVATION', 'HEIGHT_ROOF', 'height_per_squared_meter']], 
        gpd.GeoDataFrame(geometry=geodataset[f'buffer_{radius_meter}m'], crs=bv.crs), 
        predicate="intersects",     # within
        how='inner'
        ).drop_duplicates(subset=['index_right', 'geometry'])
    intersecting_buildings.columns = intersecting_buildings.columns.str.lower()
    
    grnd_elev = intersecting_buildings.groupby('index_right')['ground_elevation']
    height_roof = intersecting_buildings.groupby('index_right')['height_roof']
    height_per_squared_meter = intersecting_buildings.groupby('index_right')['height_per_squared_meter']

    geodataset[f"kml_max_grnd_elev_{radius_meter}m"] = geodataset.index.map(grnd_elev.max()).fillna(0)
    geodataset[f"kml_min_grnd_elev_{radius_meter}m"] = geodataset.index.map(grnd_elev.min()).fillna(0)
    geodataset[f"kml_mean_grnd_elev_{radius_meter}m"] = geodataset.index.map(grnd_elev.mean()).fillna(0)
    geodataset[f"kml_std_grnd_elev_{radius_meter}m"] = geodataset.index.map(grnd_elev.std()).fillna(0)
    # geodataset[f"kml_range_grnd_elev_{radius_meter}m"] = geodataset[f"kml_max_grnd_elev_{radius_meter}m"] - geodataset[f"kml_min_grnd_elev_{radius_meter}m"]
    
    geodataset[f"kml_max_height_roof_{radius_meter}m"] = geodataset.index.map(height_roof.max()).fillna(0)
    geodataset[f"kml_min_height_roof_{radius_meter}m"] = geodataset.index.map(height_roof.min()).fillna(0)
    geodataset[f"kml_mean_height_roof_{radius_meter}m"] = geodataset.index.map(height_roof.mean()).fillna(0)
    geodataset[f"kml_std_height_roof_{radius_meter}m"] = geodataset.index.map(height_roof.std()).fillna(0)
    # geodataset[f"kml_height_roof_variability_{radius_meter}m"] = (
    #     geodataset[f"kml_std_height_roof_{radius_meter}m"] / geodataset[f"kml_mean_height_roof_{radius_meter}m"]
    # ).fillna(0)

    geodataset[f"kml_max_height_per_squared_meter_{radius_meter}m"] = geodataset.index.map(height_per_squared_meter.max()).fillna(0)
    geodataset[f"kml_min_height_per_squared_meter_{radius_meter}m"] = geodataset.index.map(height_per_squared_meter.min()).fillna(0)
    geodataset[f"kml_mean_height_per_squared_meter_{radius_meter}m"] = geodataset.index.map(height_per_squared_meter.mean()).fillna(0)
    geodataset[f"kml_std_height_per_squared_meter_{radius_meter}m"] = geodataset.index.map(height_per_squared_meter.std()).fillna(0)

    
    # geodataset[f"kml_weighted_area_by_height_{radius_meter}m"] = (
    #     geodataset.index.map(
    #         intersecting_buildings.groupby('index_right').apply(lambda x: (x['height_roof'] * x['area']).sum() / x['area'].sum())
    #     ).fillna(0)
    # )

    geodataset[f"kml_total_building_volume_{radius_meter}m"] = (
        geodataset.index.map(
            intersecting_buildings.groupby('index_right').apply(lambda x: (x['height_roof'] * x['area']).sum())
        ).fillna(0)
    )
    # geodataset[f"kml_height_density_{radius_meter}m"] = geodataset[f"kml_total_building_volume_{radius_meter}m"] / geodataset[f'buffer_{radius_meter}m_area']
    
    geodataset = geodataset.drop(columns=[f'buffer_{radius_meter}m', f'buffer_{radius_meter}m_area'])
    # display(geodataset)

Radius Areas: 100%|██████████| 16/16 [00:35<00:00,  2.25s/it]


In [ ]:
geodataset = geodataset.to_crs(epsg=4326).drop(columns=['longitude', 'latitude', 'geometry'])

display(geodataset)
print(f"{geodataset.shape=}")
print(f"{ground_df.shape=}")

geodataset.to_parquet(f'../pipeline/data/processed/{MODE}/building_footprint.parquet')

,kml_max_areas_50m,kml_min_areas_50m,kml_mean_areas_50m,kml_sum_areas_50m,kml_count_buildings_50m,kml_max_grnd_elev_50m,kml_min_grnd_elev_50m,kml_mean_grnd_elev_50m,kml_std_grnd_elev_50m,kml_max_height_roof_50m,kml_min_height_roof_50m,kml_mean_height_roof_50m,kml_std_height_roof_50m,kml_max_height_per_squared_meter_50m,kml_min_height_per_squared_meter_50m,kml_mean_height_per_squared_meter_50m,kml_std_height_per_squared_meter_50m,kml_total_building_volume_50m,kml_max_areas_100m,kml_min_areas_100m,kml_mean_areas_100m,kml_sum_areas_100m,kml_count_buildings_100m,kml_max_grnd_elev_100m,kml_min_grnd_elev_100m,kml_mean_grnd_elev_100m,kml_std_grnd_elev_100m,kml_max_height_roof_100m,kml_min_height_roof_100m,kml_mean_height_roof_100m,kml_std_height_roof_100m,kml_max_height_per_squared_meter_100m,kml_min_height_per_squared_meter_100m,kml_mean_height_per_squared_meter_100m,kml_std_height_per_squared_meter_100m,kml_total_building_volume_100m,kml_max_areas_150m,kml_min_areas_150m,kml_mean_areas_150m,kml_sum_areas_150m,kml_count_buildings_150m,kml_max_grnd_elev_150m,kml_min_grnd_elev_150m,kml_mean_grnd_elev_150m,kml_std_grnd_elev_150m,kml_max_height_roof_150m,kml_min_height_roof_150m,kml_mean_height_roof_150m,kml_std_height_roof_150m,kml_max_height_per_squared_meter_150m,kml_min_height_per_squared_meter_150m,kml_mean_height_per_squared_meter_150m,kml_std_height_per_squared_meter_150m,kml_total_building_volume_150m,kml_max_areas_200m,kml_min_areas_200m,kml_mean_areas_200m,kml_sum_areas_200m,kml_count_buildings_200m,kml_max_grnd_elev_200m,kml_min_grnd_elev_200m,kml_mean_grnd_elev_200m,kml_std_grnd_elev_200m,kml_max_height_roof_200m,kml_min_height_roof_200m,kml_mean_height_roof_200m,kml_std_height_roof_200m,kml_max_height_per_squared_meter_200m,kml_min_height_per_squared_meter_200m,kml_mean_height_per_squared_meter_200m,kml_std_height_per_squared_meter_200m,kml_total_building_volume_200m,kml_max_areas_250m,kml_min_areas_250m,kml_mean_areas_250m,kml_sum_areas_250m,kml_count_buildings_250m,kml_max_grnd_elev_250m,kml_min_grnd_elev_250m,kml_mean_grnd_elev_250m,kml_std_grnd_elev_250m,kml_max_height_roof_250m,kml_min_height_roof_250m,kml_mean_height_roof_250m,kml_std_height_roof_250m,kml_max_height_per_squared_meter_250m,kml_min_height_per_squared_meter_250m,kml_mean_height_per_squared_meter_250m,kml_std_height_per_squared_meter_250m,kml_total_building_volume_250m,kml_max_areas_275m,kml_min_areas_275m,kml_mean_areas_275m,kml_sum_areas_275m,kml_count_buildings_275m,kml_max_grnd_elev_275m,kml_min_grnd_elev_275m,kml_mean_grnd_elev_275m,kml_std_grnd_elev_275m,kml_max_height_roof_275m,kml_min_height_roof_275m,kml_mean_height_roof_275m,kml_std_height_roof_275m,kml_max_height_per_squared_meter_275m,kml_min_height_per_squared_meter_275m,kml_mean_height_per_squared_meter_275m,kml_std_height_per_squared_meter_275m,kml_total_building_volume_275m,kml_max_areas_300m,kml_min_areas_300m,kml_mean_areas_300m,kml_sum_areas_300m,kml_count_buildings_300m,kml_max_grnd_elev_300m,kml_min_grnd_elev_300m,kml_mean_grnd_elev_300m,kml_std_grnd_elev_300m,kml_max_height_roof_300m,kml_min_height_roof_300m,kml_mean_height_roof_300m,kml_std_height_roof_300m,kml_max_height_per_squared_meter_300m,kml_min_height_per_squared_meter_300m,kml_mean_height_per_squared_meter_300m,kml_std_height_per_squared_meter_300m,kml_total_building_volume_300m,kml_max_areas_350m,kml_min_areas_350m,kml_mean_areas_350m,kml_sum_areas_350m,kml_count_buildings_350m,kml_max_grnd_elev_350m,kml_min_grnd_elev_350m,kml_mean_grnd_elev_350m,kml_std_grnd_elev_350m,kml_max_height_roof_350m,kml_min_height_roof_350m,kml_mean_height_roof_350m,kml_std_height_roof_350m,kml_max_height_per_squared_meter_350m,kml_min_height_per_squared_meter_350m,kml_mean_height_per_squared_meter_350m,kml_std_height_per_squared_meter_350m,kml_total_building_volume_350m,kml_max_areas_400m,kml_min_areas_400m,kml_mean_areas_400m,kml_sum_areas_400m,kml_count_buildings_400m,kml_max_grnd_elev_400m,kml_min_grnd_elev_400m,kml_mean_grnd_elev_400m,kml_s

geodataset.shape=(1040, 288)
ground_df.shape=(1040, 3)
